In [1]:
import numpy as np
import pandas as pd
from sklearn import tree,linear_model,neural_network
from sklearn.neural_network import MLPClassifier
import re

In [2]:
train= pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
predictions = pd.read_csv("gender_submission.csv")

In [3]:
labels=train.Survived
train.drop(columns=['Survived'],inplace=True)

In [4]:
join=pd.concat([train,test])

In [5]:
join.drop(columns=['PassengerId','Cabin','Fare','Ticket'],inplace=True)

In [6]:
Sex = pd.get_dummies(join.Sex)
Sex = Sex.male
join.Sex = Sex

In [7]:
Embarked = pd.get_dummies(join.Embarked)
Embarked = Embarked.iloc[:,0:2]
join = pd.concat([join,Embarked],axis = 1)
join.drop(['Embarked'],axis = 1,inplace = True)

In [8]:
pclass=pd.get_dummies(join.Pclass)
pclass=pclass.iloc[:,0:2]
join=pd.concat([join,pclass],axis=1)
join.drop(columns=['Pclass'],inplace=True)

In [9]:
lis=[]
for i in join.Name:
    start = re.escape(", ")
    end   = re.escape(". ")
    result = re.search('%s(.*)%s' % (start, end), i).group(1)
    lis.append(result)

join.Name=lis

In [10]:
for i in range(len(join)):
    if join.Name.values[i]=='Don' or join.Name.values[i]=='Dr' or join.Name.values[i]=='Major' or join.Name.values[i]=='Rev' or join.Name.values[i]=='Col' or join.Name.values[i]=='Capt' or join.Name.values[i]=='Dona' or join.Name.values[i]=='Jonkheer' or join.Name.values[i]=='the Countess' or join.Name.values[i]=='Sir':
        join.Name.values[i]='Mr'
    if join.Name.values[i]=='Lady' or join.Name.values[i]=='Mlle' or join.Name.values[i]=='Ms' or join.Name.values[i]=='Mme' or join.Name.values[i]=='Mrs. Martin (Elizabeth L':
        join.Name.values[i]='Mrs'

In [11]:
dic={'Mr':1,'Mrs':2,'Miss':3,'Master':4}
join.Name=join.Name.map(dic)

In [12]:
dic1={1:join.groupby(['Name']).Age.mean()[1],2:join.groupby(['Name']).Age.mean()[2],3:join.groupby(['Name']).Age.mean()[3],4:join.groupby(['Name']).Age.mean()[4],}


In [13]:
join.Age.fillna(value=join.Name.map(dic1),inplace=True)

In [14]:
join['Family']=0
for i in range(len(join)):
    join.Family.values[i]=join.SibSp.values[i]+join.Parch.values[i]

In [15]:
for i in range(len(join)):
    if join.Family.values[i]>0:
        join.Family.values[i]=1

In [16]:
join.drop(columns=['Name'],inplace=True)

In [17]:
join.corr()

,Sex,Age,SibSp,Parch,C,Q,1,2,Family
Sex,1.000000,0.086519,-0.109609,-0.213125,-0.066564,-0.088651,-0.107371,-0.028862,-0.284537
Age,0.086519,1.000000,-0.221898,-0.140485,0.081809,-0.044880,0.369295,-0.013146,-0.133672
SibSp,-0.109609,-0.221898,1.000000,0.373587,-0.048396,-0.048678,-0.034256,-0.052419,0.591077
Parch,-0.213125,-0.140485,0.373587,1.000000,-0.008635,-0.100943,-0.013033,-0.010057,0.549022
C,-0.066564,0.081809,-0.048396,-0.008635,1.000000,-0.164166,0.325722,-0.134675,0.107874
Q,-0.088651,-0.044880,-0.048678,-0.100943,-0.164166,1.000000,-0.166101,-0.121973,-0.127214
1,-0.107371,0.369295,-0.034256,-0.013033,0.325722,-0.166101,1.000000,-0.296526,0.126551
2,-0.028862,-0.013146,-0.052419,-0.010057,-0.134675,-0.121973,-0.296526,1.000000,0.035075
Family,-0.284537,-0.133672,0.591077,0.549022,0.107874,-0.127214,0.126551,0.035075,1.000000


In [18]:
x_test=join.iloc[891: :, :]
x_cv=join.iloc[801:891:, :]
x_train=join.iloc[ :801:, :]

In [19]:
y_cv=labels.iloc[801:]
y_train=labels.iloc[:801]

In [20]:
clf1=MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(18), random_state=0)
clf1.fit(x_train,y_train)
train_accuracy1=clf1.score(x_cv,y_cv)
train_accuracy1

0.8888888888888888

In [21]:
y_pred1=clf1.predict(x_test)


In [22]:
predictions['Survived']=y_pred1
predictions.to_csv("pred.csv",index = False)